In [2]:
# imports and setup
import pandas as pd
# lemmitization
import spacy
# python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
# pure stemming
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

import wordcloud
import matplotlib.pyplot as plt

from pandas import json_normalize



## Data Collection and Preprocessing:

In [3]:
# read json data tp json_data
json_data = pd.read_json('data/dev-v1.1.json')

# Use json_normalize to flatten question and id, while keeping answers
df = json_normalize(
    json_data['data'], 
    record_path=['paragraphs', 'qas'], 
    meta=['title', ['paragraphs', 'context']], 
    errors='ignore'
)

# Extract answers and create separate columns for answer1, answer2, answer3
df[['answer1', 'answer2', 'answer3']] = pd.DataFrame(
    df['answers'].apply(lambda ans: [answer['text'] for answer in ans[:3]]).to_list(), index=df.index
)

# Drop the original 'answers' column
df = df.drop(columns=['answers'])

# Display the result
df.head()

,question,id,title,paragraphs.context,answer1,answer2,answer3
0,Which NFL team represented the AFC at Super Bo...,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Denver Broncos,Denver Broncos,Denver Broncos
1,Which NFL team represented the NFC at Super Bo...,56be4db0acb8001400a502ed,Super_Bowl_50,Super Bowl 50 was an American football game to...,Carolina Panthers,Carolina Panthers,Carolina Panthers
2,Where did Super Bowl 50 take place?,56be4db0acb8001400a502ee,Super_Bowl_50,Super Bowl 50 was an American football game to...,"Santa Clara, California",Levi's Stadium,Levi's Stadium in the San Francisco Bay Area a...
3,Which NFL team won Super Bowl 50?,56be4db0acb8001400a502ef,Super_Bowl_50,Super Bowl 50 was an American football game to...,Denver Broncos,Denver Broncos,Denver Broncos
4,What color was used to emphasize the 50th anni...,56be4db0acb8001400a502f0,Super_Bowl_50,Super Bowl 50 was an American football game to...,gold,gold,gold
